In [8]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [9]:
df = pd.read_csv('../data/df_high_ratio.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595 entries, 0 to 1594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1595 non-null   object 
 1   text                1595 non-null   object 
 2   target              1595 non-null   object 
 3   special_char_count  1595 non-null   int64  
 4   special_char_ratio  1595 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 62.4+ KB


,ID,text,target,special_char_count,special_char_ratio
0,ynat-v1_train_00624,"사건!>실""를 b$#라 #극)]체O.;월f:?",생활문화,14,0.560000
1,ynat-v1_train_02413,"전W;참c 이nd 유j]""m객ie((우려` ?<. 개조",세계,16,0.533333
2,ynat-v1_train_01203,nO통령z\후 p싱턴으로 ^R!與지p부 @re-#{{\,정치,15,0.500000
3,ynat-v1_train_02765,"$G ;!p 서울 |곳 나vo객 =적""bs 귀s,d거)",사회,15,0.500000
4,ynat-v1_train_02275,더*} ]i대+ 김현권!v원a}_대8*vdL줄!알았d,정치,15,0.500000
5,ynat-v1_train_02105,"^r^[P홀w2019년H'v요?서트vYp 세,#uNs[cJ",생활문화,16,0.500000
6,ynat-v1_train_01383,H@마 대통령uir6후m워싱턴_(ymy무( 개k_z정,세계,14,0.482759
7,ynat-v1_train_02013,에+-W[성엘에스티에bm00yxr규$ -:,경제,11,0.478261
8,ynat-v1_train_01228,디딤. 대;에도g모-#신> !Om*,경제,9,0.473684
9,ynat-v1_train_00247,m시판 코.아 Vvg페ogt,IT과학,7,0.466667


In [10]:
test_df = pd.read_csv('../data/noise_free_data.csv')
test_df.info()
test_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1205 non-null   object 
 1   text                1205 non-null   object 
 2   target              1205 non-null   object 
 3   special_char_count  1205 non-null   int64  
 4   special_char_ratio  1205 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 47.2+ KB


,ID,text,target,special_char_count,special_char_ratio
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,경제,0,0.0
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자 연인 빚 떠안은 적 있다,생활문화,0,0.0
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파 3위 굳,IT과학,0,0.0
3,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요 경주 지진현장 방문종합,세계,0,0.0
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔 알뜰폰 기대감,IT과학,0,0.0
5,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,스포츠,0,0.0
6,ynat-v1_train_00012,황총리 각 부처 비상대비태세 철저히 강구해야,정치,0,0.0
7,ynat-v1_train_00015,게시판 KISA 박민정 책임연구원 APTLD 이사 선출,생활문화,0,0.0
8,ynat-v1_train_00016,공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예,사회,0,0.0
9,ynat-v1_train_00017,월세 전환에 늘어나는 주거비 부담 작년 역대 최고치,경제,0,0.0


In [11]:
# 'ID', 'text', 'target' 열을 제외하고 나머지 열 제거
df = df[['ID', 'text', 'target']]

# 결과 확인
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595 entries, 0 to 1594
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      1595 non-null   object
 1   text    1595 non-null   object
 2   target  1595 non-null   object
dtypes: object(3)
memory usage: 37.5+ KB
None


In [12]:
target_counts = df['target'].value_counts().sort_index()
print(target_counts)

target
IT과학    227
경제      230
사회      226
생활문화    227
세계      230
스포츠     227
정치      228
Name: count, dtype: int64


In [13]:
# 데이터프레임에서 텍스트와 라벨 선택
X = df['text']  # 텍스트 데이터
y = df['target']  # 라벨 데이터

# 데이터 분할 (훈련/테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# 분류 모델 훈련 (Naive Bayes 사용)
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

In [ ]:
# 예측
y_pred = model.predict(X_test_tfidf)

In [ ]:
# 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))